<a href="https://colab.research.google.com/github/davidsonsantana89/dass--CESAR_SCHOOL-Topicos_Contemporaneos/blob/main/exercicios/parte_2/dass_08_LangChain_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAVIDSON ALVES SANTOS DE SANTANA

dass@cesar.school

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [8]:
!pip install langchain_openai langchain_core

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-c0196bf6-07b1-418d-84d0-9c1154af23bb-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


In [7]:
response.content

'Olá! Como posso ajudar você hoje?'

## Prompts

### Templates Simples

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

# Runnable
prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [11]:
response = llm.invoke(prompt)

print(response.content)

A inteligência artificial é o futuro!


In [13]:
prompt_template = ChatPromptTemplate.from_template("Qual a cor do objeto {obj}? Responda apenas com uma palavra indicando a cor.")

# Runnable
prompt = prompt_template.invoke({"obj": "limão"})
print(prompt)

response = llm.invoke(prompt)

print(response.content)

messages=[HumanMessage(content='Qual a cor do objeto limão? Responda apenas com uma palavra indicando a cor')]
Amarelo.


### Templates de Mensagens

In [14]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-fe57c1ad-2c7b-4272-bf13-49b96c48bd63-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [16]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [17]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [18]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [19]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [20]:
chain = prompt_template | llm | parser

In [21]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [37]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [23]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [24]:
prompt_template = ChatPromptTemplate.from_template("Faça uma piada sobre {topico}")

chain = prompt_template | llm | parser

chain.invoke({"topico": "futebol"})

'Por que o jogador de futebol levou um lápis para o jogo?\n\nPorque ele queria fazer um gol de letra! ⚽✏️'

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [31]:
prompt_template = ChatPromptTemplate.from_template("Classifique o sentimento em positivo, neutro ou negativo do {topico}")

chain = prompt_template | llm | parser

# chain.invoke({"topico": "O dia está maravilhoso"})
chain.invoke({"topico": "Que pena que tudo acabou entre nós dois."})

'A frase "Que pena que tudo acabou entre nós dois" expressa um sentimento negativo. A palavra "pena" indica tristeza ou arrependimento em relação ao término da relação.'

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [33]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um programador que converte descrições de funções em código Python."),
        ("user", "{texto}")
    ]
)

chain = prompt_template | llm | parser

result = chain.invoke({"texto": "uma função que calcule a soma de dois números"})

print(result)

Aqui está uma função em Python que calcula a soma de dois números:

```python
def soma(a, b):
    return a + b
```

Você pode usar essa função da seguinte maneira:

```python
resultado = soma(5, 3)
print(resultado)  # Saída: 8
```


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [44]:
prompt_template_explicacao = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um especialista em conhecimentos gerais que explica tópicos de maneira simplificada para o usuário."),
        ("user", "{texto}")
    ]
)


prompt_template_tradutor = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de textos do português para o inglês."),
        ("user", "{texto}")
    ]
)

chain = prompt_template_explicacao | llm | parser | prompt_template_tradutor | llm | parser

result = chain.invoke({"texto": "um poliedro"})

print(result)

A polyhedron is a three-dimensional shape that has flat faces, edges, and vertices. The faces are the sides of the polyhedron, the edges are the lines where two faces meet, and the vertices are the points where the edges meet.

Polyhedra can be classified into different types, such as:

1. **Convex Polyhedra**: All of their faces "point outward," and any line connecting two points inside the polyhedron lies entirely within it.

2. **Concave Polyhedra**: Have at least one face that "points inward," meaning there are points that, if connected, can extend outside the polyhedron.

The most common examples of polyhedra include the cube (which has 6 square faces) and the pyramid (which can have different shapes for its base).

In summary, polyhedra are solids that have many faces, and each face is a flat shape!


### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [45]:
llm.invoke("O que é a Cesar School?")

AIMessage(content='A Cesar School é uma instituição de ensino que faz parte do grupo CESAR (Centro de Estudos e Sistemas Avançados do Recife). Localizada em Recife, Pernambuco, a escola é voltada para a formação de profissionais nas áreas de tecnologia, inovação e design. Oferece uma variedade de cursos, incluindo programas de graduação, pós-graduação e cursos livres, com foco em tópicos como desenvolvimento de software, design de interação, inteligência artificial e outras áreas relacionadas à transformação digital.\n\nA Cesar School é conhecida por seu enfoque prático e pela conexão com o mercado, buscando preparar os alunos para os desafios do mundo profissional. Além disso, a escola frequentemente colabora com empresas e organizações para promover projetos e iniciativas que incentivem a inovação e o empreendedorismo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 14, 'total_tokens': 166}, 'model_name': 'gpt-4o-mi